In [1]:
%load_ext autoreload
%autoreload 2

from app.core.config import settings
from app.utils.object_storage import ObjectStorage
from app.services.nlp import QAService
from app.core.logging import FileLogger, MongoDBLogger,DBLogger

from pydantic import BaseModel, Field
from pydantic_settings import BaseSettings
from typing import Optional

In [ ]:
curl -X 'POST' \
  'http://localhost:8000/api/v1/qa/answer' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "question": "what is capital of france",
  "context": "paris is capital of france"
}'

In [1]:
import requests

In [ ]:
url = "http://localhost:8000/api/v1/qa/answer"
data = {
    "question": "What is the capital of France?",
    "context": "Paris is the capital of France."
}
requests.post(url, json=data)

In [ ]:
for i in range(200):
    requests.post(url, json=data)

In [ ]:
settings.model_dump()

In [19]:
p_dict= {
    "model_name": "qa_model",
    "model_tag": "v1",
    "model_version": "1.0",
    "name":"qa_model-v1-1-0",
}

In [ ]:
db_logger = DBLogger(logger=settings.LOGGER_HANDLER,**p_dict)

In [ ]:
db_logger.log_operation("test")

In [ ]:
qa = QAService()

In [ ]:
qa.predict(question="What is the capital of France?", context="Paris and Madrid are both of capitals in Europe. Madrid is the capital of Spain.")

In [ ]:
qa.name

In [ ]:
settings

In [ ]:
settings.LOGGER_HANDLER

In [ ]:
settings.LOG_DIR

In [18]:
logger = MongoDBLogger(db_name="test", model_name="test", model_tag="test")

In [ ]:
logger.log_operation("test_meesage")

In [ ]:
class QAService:
    def __init__(self, db_logger_wrapper: DBLoggerWrapper, settings: Any = None):
        self.settings = settings
        self.model = None
        self.model_name = self.settings.QA_NAME
        self.model_tag = self.settings.QA_TAG
        
        self.db_logger = db_logger_wrapper
        
        self.load_model()
        self.model.eval()

    def load_model(self):
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.settings.QA_MODEL_PATH)
            self.model = AutoModelForQuestionAnswering.from_pretrained(self.settings.QA_MODEL_PATH)
            self.db_logger.log_operation(f"Model loaded successfully: {self.settings.QA_MODEL_PATH}")
            self.db_logger.log_metadata({
                "model_path": self.settings.QA_MODEL_PATH,
                "tokenizer_type": type(self.tokenizer).__name__,
                "model_type": type(self.model).__name__
            })
        except Exception as e:
            self.db_logger.log_operation(f"Error loading model: {str(e)}", "error", exc_info=True)
            raise

    def answer_question(self, question: str, context: str):
        try:
            inputs = self.tokenizer(question, context, return_tensors="pt")

            with torch.no_grad():
                outputs = self.model(**inputs)

            start_scores = outputs.start_logits
            end_scores = outputs.end_logits

            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores)

            answer_tokens = inputs["input_ids"][0][start_index : end_index + 1]
            answer = self.tokenizer.decode(answer_tokens)

            result = {"answer": answer, "start": int(start_index), "end": int(end_index)}
    
            self.db_logger.log_model_results({
                "question_length": len(question),
                "context_length": len(context),
                "answer_length": len(answer)
            })

            return result
        except Exception as e:
            self.db_logger.log_operation(f"Error answering question: {str(e)}", "error", exc_info=True)
            raise

    def predict(self, question: str, context: str):
        return self.answer_question(question, context)

    def get_prediction(self, prediction_id: str):
        raise NotImplementedError("This method is not implemented in the QAService class.")
        # return self.db_logger.get_prediction(prediction_id)

    def get_model_performance(self, start_date: datetime, end_date: datetime):
        return self.db_logger.get_model_performance(start_date, end_date)


In [ ]:
logger.log_operation("This is a test log message.", level="info")

In [ ]:
self._store_prediction(question, context, result)

In [ ]:
try:
            inputs = self.tokenizer(question, context, return_tensors="pt")

            with torch.no_grad():
                outputs = self.model(**inputs)

            start_scores = outputs.start_logits
            end_scores = outputs.end_logits

            start_index = torch.argmax(start_scores)
            end_index = torch.argmax(end_scores)

            answer_tokens = inputs["input_ids"][0][start_index : end_index + 1]
            answer = self.tokenizer.decode(answer_tokens)

            result = {"answer": answer, "start": int(start_index), "end": int(end_index)}

            self.db_logger.log_model_results(
                input_info={"question": question, "context": context},
                results={"question_length": len(question), "context_length": len(context), "answer_length": len(answer)}
            )

            return result
        except Exception as e:
            self.db_logger.log_operation(f"Error answering question: {str(e)}", "error", exc_info=True)
            raise


In [17]:
class ModelSettings(BaseSettings):
    MODEL_NAME: str = "model_name"  
    MODEL_VERSION: str = "model_version"



 model_config = ModelSettings(MODEL_NAME=settings.QA_NAME)   

In [ ]:
qa.storage.store_object?

In [ ]:
qa = QAService()

In [ ]:
qa.predict("What is the capital of France?", "France is a country in Europe.")

In [ ]:
qa.answer_question("What is the capital of France?", "Paris is the capital of France.")

In [3]:
client = ObjectStorage(endpoint=settings.MINIO_ENDPOINT, access_key=settings.MINIO_ACCESS_KEY, secret_key=settings.MINIO_SECRET_KEY, secure=False)

In [34]:
from app.core.config import settings,load_config_yaml
from pydantic_settings import BaseSettingsaa

In [ ]:
from pydantic_settings import (
    BaseSettings,
    SettingsConfigDict,
    PydanticBaseSettingsSource,
    YamlConfigSettingsSource
)
from pydantic import Field
from typing import Tuple, Type


class Settings(BaseSettings):
    # Define your settings here with default values
    PROJECT_NAME: str = Field(default="ML API_0")
    PROJECT_VERSION: str = Field(default="0.0.0")
    PORT: int = Field(default=8000)
    
    
    # CLIP_ENABLE: bool = Field(default=False)
    # QA_ENABLE: bool = Field(default=False)

    # # Add other settings as needed
    # CLIP_MODEL_PATH: str = Field(default="")
    # QA_MODEL_PATH: str = Field(default="")
    

    model_config = SettingsConfigDict(
        env_file='.env',
        env_file_encoding='utf-8',
        extra="allow",
        yaml_file="config/config.yaml",
        case_sensitive=True
    )

    @classmethod
    def settings_customise_sources(
        cls,
        settings_cls: Type[BaseSettings],
        init_settings: PydanticBaseSettingsSource,
        env_settings: PydanticBaseSettingsSource,
        dotenv_settings: PydanticBaseSettingsSource,
        file_secret_settings: PydanticBaseSettingsSource,
    ) -> Tuple[PydanticBaseSettingsSource, ...]:
        
        return (      
           # env_settings,
          #  file_secret_settings,
            dotenv_settings,
            YamlConfigSettingsSource(settings_cls),
            init_settings,
            
        )
# Instantiate the settings
Settings().model_dump()

In [ ]:
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field, validator
from omegaconf import OmegaConf
from typing import Dict, Any
import os

class Settings(BaseSettings):
    CONFIG_PATH: str = Field(default="config/config.yaml")

    # Define your settings fields here. These will be populated from the YAML file.
    PROJECT_NAME: str = Field(default="ML API_0")
    PROJECT_VERSION: str = Field(default="0.0.0")
    CLIP_ENABLE: bool = Field(default=False)
    QA_ENABLE: bool = Field(default=False)

    # Add other fields as needed
    # CLIP_MODEL_PATH: str
    # QA_MODEL_PATH: str
    # PORT: int

    model_config = SettingsConfigDict(
        env_file=".env",
        env_file_encoding="utf-8",
        extra="allow",
    )

    @validator("*", pre=True)
    def load_yaml_config(cls, v, field):
        config_path = os.getenv("CONFIG_PATH", cls.CONFIG_PATH)
        if os.path.exists(config_path):
            config = OmegaConf.load(config_path)
            if field.name in config:
                return config[field.name]
        return v

    class Config:
        validate_assignment = True

settings = Settings()

In [ ]:
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field
from omegaconf import OmegaConf
from typing import Dict, Any
import os

CONFIG_YAML_PATH = "config/config.yaml"

def load_config_yaml() -> Dict[str, Any]:
    if os.path.exists(CONFIG_YAML_PATH):
        config = OmegaConf.load(CONFIG_YAML_PATH)
        return OmegaConf.to_container(config, resolve=True)
    return {}

class Settings(BaseSettings):
    PROJECT_NAME: str = Field(default="ML API_0")
    PROJECT_VERSION: str = Field(default="0.0.0")
    CLIP_ENABLE: bool = Field(default=False)
    QA_ENABLE: bool = Field(default=False)

    # Add other fields as needed
    # CLIP_MODEL_PATH: str = Field(default="ml/siglip-base-patch16-224.xml")
    # QA_MODEL_PATH: str = Field(default="distilbert-base-cased-distilled-squad")
    # PORT: int = Field(default=8000)

    # model_config = SettingsConfigDict(
    #     env_file=".env",
    #     env_file_encoding="utf-8",
    #     extra="allow",
    # )

    @classmethod
    def settings_customise_sources(
        cls,
     
        init_settings,
        env_settings,
        file_secret_settings,
    ):
        yaml_settings = load_config_yaml()
        return (
            init_settings,
            #yaml_settings,
            env_settings,
            file_secret_settings,
        )


settings = Settings()

In [ ]:
settings

In [12]:
res = getattr(settings,"PORT")

In [14]:
import yaml
from typing import Dict, Any
from pydantic_settings import BaseSettings
from pydantic import Field

def yaml_config_settings_source(settings: BaseSettings) -> Dict[str, Any]:
    """
    A simple settings source that loads variables from a YAML file
    at the project's root.
    """
    with open("config.yaml", "r") as yaml_file:
        return yaml.safe_load(yaml_file)

class Settings(BaseSettings):
    PROJECT_NAME: str = Field(default="ML API")
    PROJECT_VERSION: str = Field(default="1.0.0")

    # Model paths
    CLIP_MODEL_PATH: str = Field(default="ml/siglip-base-patch16-224.xml")
    QA_MODEL_PATH: str = Field(default="distilbert-base-cased-distilled-squad")

    # Feature flags
    CLIP_ENABLE: bool = Field(default=False)
    QA_ENABLE: bool = Field(default=False)

    PORT: int = Field(default=8000)

    # MinIO settings
    MINIO_ENDPOINT: str = Field(default="minio:9000")
    MINIO_ACCESS_KEY: str = Field(default="minioadmin")
    MINIO_SECRET_KEY: str = Field(default="minioadmin")
    MINIO_SECURE: bool = Field(default=False)

    class Config:
        env_file = ".env"
        env_file_encoding = 'utf-8'
        extra = 'ignore'

        @classmethod
        def customise_sources(
            cls,
            init_settings,
            env_settings,
            file_secret_settings,
        ):
            return (
                init_settings,
                yaml_config_settings_source,
                env_settings,
                file_secret_settings,
            )

settings = Settings()

In [ ]:
a.upper()

In [2]:
import numpy as np
import torch

In [ ]:
from transformers import pipeline

In [ ]:
model_name = "distilbert-base-cased-distilled-squad"

pipeline(model=model_name, tokenizer=model_name, task='question-answering')

In [ ]:
model_name = "distilbert-base-cased-distilled-squad"

model = pipeline(model=model_name, tokenizer=model_name, task='question-answering')

In [ ]:
model("is football sport ?", "Football is a family of team sports that involve, to varying degrees, kicking a ball to score a goal. Unqualified, the word football generally means the form of football that is the most popular where the word is used.")

In [1]:
from app.services.nlp import QAService

In [ ]:
qa = QAService()


In [ ]:
qa.answer_question("What is the capital of France?","Football is a team sport played between two teams of eleven players with a spherical ball. It is played by 250 million players in over 200 countries and dependencies, making it the world's most popular sport.")